# Masks not Comprehension lists for Numpy Arrays

Masks are are more efficient to use numpy arrays than list comprehension.
Here I will show two places I have improved speed in inherited code.

The first example is a function that selects a section out of a spectra specified by wavelength range. 

In [86]:
import numpy as np
from __future__ import division

In [28]:
def wav_selector(wav, flux, wav_min, wav_max):
    """
    function that returns wavelength and flux within a given range
    """    
    wav_sel = np.array([value for value in wav if(wav_min < value < wav_max)], dtype="float64")
    flux_sel = np.array([value[1] for value in zip(wav,flux) if(wav_min < value[0] < wav_max)], dtype="float64")
    
    return [wav_sel, flux_sel]

In [29]:
Test_wav = np.linspace(2000,2200,300000)  # nm
Test_flux = np.random.random(len(Test_wav)) # nm
min_wav = 2050
max_wav = 2170

# If you had lists instead
Test_list_wav = list(Test_wav)
Test_list_flux = list(Test_flux)

In [30]:
# Timing it running
print("wav_selector with numpy inputs")
%timeit wav_selector(Test_wav, Test_flux, min_wav, max_wav)
print("\nwav_selector list inputs")
%timeit wav_selector(Test_list_wav, Test_list_flux, min_wav, max_wav)

wav_selector with numpy inputs
1 loop, best of 3: 253 ms per loop

wav_selector list inputs
1 loop, best of 3: 215 ms per loop


This is intesting that inputing lists is faster in this function than numpy arrays. Numpy arrays are built to be faster.

Looking at the code in wav_selector we see it preforms a comphension list and turns it back into a numpy array. 

It also actively turns the input values into an array even though you could pass it lists.

I have changed the function to use masks on the numpy arrays and to do list comprehension on lists. This avoids any time spent changing between the different data types.


In [31]:
# wav_selector using 
def fast_wav_selector(wav, flux, wav_min, wav_max):
    """ Faster Wavelength selector
    If passed lists it will return lists.
    If passed np arrays it will return arrays
    """
  
    if isinstance(wav, list): # if passed lists
          wav_sel = [value for value in wav if(wav_min < value < wav_max)]
          flux_sel = [value[1] for value in zip(wav,flux) if(wav_min < value[0] < wav_max)]
    
    elif isinstance(wav, np.ndarray):
        # Super Fast masking with numpy
        mask = (wav > wav_min) & (wav < wav_max)
        wav_sel = wav[mask]
        flux_sel = flux[mask]
    else:
          raise TypeError("Unsupported input wav type")
    return [wav_sel, flux_sel]

In [32]:
print("fast_wav_selector with numpy inputs")
%timeit fast_wav_selector(Test_wav, Test_flux, min_wav, max_wav)

print("\nfast_wav_selector list inputs")
%timeit fast_wav_selector(Test_list_wav, Test_list_flux, min_wav, max_wav)

fast_wav_selector with numpy inputs
1000 loops, best of 3: 970 µs per loop

fast_wav_selector list inputs
1 loop, best of 3: 195 ms per loop


We can see here that the numpy mask is about >100 X faster than the old version.

Also when you input lists it runs slightly faster ~15% for this case.

# Convolution Example 

Here is another example that is part of a convolution to a desired Resolution(R). 
It gets ran for every wavelength value (wav) in the spectra which can be over 100,000 values. It was a large bottleneck in my research due to the comprehension lists. 

Lets see the preformance increase with masks.

In [87]:
# Other function we need
def unitary_Gauss(x, center, FWHM):
    """
    Gaussian_function of area=1
	
	p[0] = A;
	p[1] = mean;
	p[2] = FWHM;
    """
    
    sigma = np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) );
    Amp = 1.0 / (sigma*np.sqrt(2*np.pi))
    tau = -((x - center)**2) / (2*(sigma**2))
    result = Amp * np.exp( tau );
    
    return result

In [96]:
# This is the inner loop 
def convolve(wav, R, wav_extended, flux_extended, FWHM_lim):
        FWHM = wav/R
        
        indexes = [ i for i in range(len(wav_extended)) if ((wav - FWHM_lim*FWHM) < wav_extended[i] < (wav + FWHM_lim*FWHM))]

        flux_2convolve = flux_extended[indexes[0]:indexes[-1]+1]
        IP = unitary_Gauss(wav_extended[indexes[0]:indexes[-1]+1], wav, FWHM)
        
        val = np.sum(IP*flux_2convolve) 
        unitary_val = np.sum(IP*np.ones_like(flux_2convolve))  # Effect of convolution onUnitary. For changing number of points
        
        return val/unitary_val

In [97]:
# This is the improved version with masks
def fast_convolve(wav, R, wav_extended, flux_extended, FWHM_lim):
    FWHM = wav/R
    # Numpy mask
    index_mask = (wav_extended > (wav - FWHM_lim*FWHM)) &  (wav_extended < (wav + FWHM_lim*FWHM))
    
    flux_2convolve = flux_extended[index_mask]
    IP = unitary_Gauss(wav_extended[index_mask], wav, FWHM)
    
    val = np.sum(IP*flux_2convolve) 
    unitary_val = np.sum(IP*np.ones_like(flux_2convolve))  # Effect of convolution onUnitary. For changing number of points
        
    return val/unitary_val

In [109]:
# some Random test spectra
wav_extended = np.linspace(2020,2040,10000)
FWHM_lim=5
R = 500000

flux_extended = np.random.random(len(wav_extended)) + np.ones_like(wav_extended)

wav = 2029. # one wave length value

In [110]:
print("Convolution with comprehension list and indices")
%timeit convolve(wav, R, wav_extended, flux_extended, FWHM_lim)
print("Faster convolution with masks")
%timeit fast_convolve(wav, R, wav_extended, flux_extended, FWHM_lim)

Convolution with comprehension list and indices
100 loops, best of 3: 2.33 ms per loop
Faster convolution with masks
10000 loops, best of 3: 31.6 µs per loop


That was for one time though the loop with a large differnce in speed. 

Now lets loop this over all the wavelength values and see the change in time of result

I will also compare the affect of saving the result from each loop as a list or in a pre allocated numpy array.

In [112]:
%%timeit 
# Preallocating a numpy array is also faster than appending to a list
flux_conv_res = np.empty_like(wav_extended)
#print("Convolution with comprehension list and indices")
for i, wav in enumerate(wav_extended):
    flux_conv_res[i] = convolve(wav, R, wav_extended, flux_extended, FWHM_lim)


1 loop, best of 3: 47.4 s per loop


In [113]:
%%timeit 
# Preallocating a numpy array is also faster than appending to a list 
flux_conv_res = np.empty_like(wav_extended)
#print("Faster convolution with masks")
for i, wav in enumerate(wav_extended):
    flux_conv_res = fast_convolve(wav, R, wav_extended, flux_extended, FWHM_lim)

1 loop, best of 3: 332 ms per loop


47s for comprehension verse 300ms for masks is around 150 times faster

Shown below it is also much faster to preallocate the numpy array and then fill it up rather than to appending to the list every time.

In [ ]:
%%timeit 
# Preallocating a numpy array is also faster than appending to a list in each loop but I won't show that now.
flux_conv_res = []
#print("Convolution with comprehension list and indices")
for i, wav in enumerate(wav_extended):
    flux_conv_res.append(convolve(wav, R, wav_extended, flux_extended, FWHM_lim)) 

In [ ]:
%%timeit 
flux_conv_res = []
#print("Convolution with comprehension list and indices")
for i, wav in enumerate(wav_extended):
    flux_conv_res.append(fast_convolve(wav, R, wav_extended, flux_extended, FWHM_lim)) 